<a href="https://colab.research.google.com/github/windopper/NerualNetworkPracticeInJupyter/blob/main/VectorSimilarity/cosin_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 코사인 유사도 (Cosine Similarity)

코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미함 두 벡터의 방향이 완전히 동일한 경우에는 1의 값을 가지며, 90°의 각을 이루면 0, 180°로 반대의 방향을 가지면 -1의 값을 갖게 됨. 즉, 결국 코사인 유사도는 -1 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단할 수 있음 이를 직관적으로 이해하면 두 벡터가 가리키는 방향이 얼마나 유사한가를 의미함

Numpy를 사용한 코사인 유사도 계산

In [3]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
  return dot(a, b)/(norm(a)*norm(b))

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서 3의 유사도 :', cos_sim(doc1, doc3))
print('문서 2와 문서 3의 유사도 :', cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서 3의 유사도 : 0.6666666666666667
문서 2와 문서 3의 유사도 : 1.0000000000000002


# 유사도를 이용한 추천 시스템 구현

In [8]:
import pandas as pd
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

uploaded = files.upload()


data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

Saving movies_metadata.csv to movies_metadata.csv


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [21]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)

In [22]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)

In [23]:
print('overview 열의 결측값의 수 :', data['overview'].isnull().sum())

overview 열의 결측값의 수 : 135


null값을 pandas의 fillna()를 사용하여 empty value로 대체

In [24]:
data['overview'] = data['overview'].fillna('')

overview 열에 대해서 TF-IDF 행렬을 구한후 행렬의 크기 출력

In [25]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


20000개의 영화를 표현하기 위하여 47487개의 단어가 사용됨

In [26]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


코사인 유사도 연산을 통하여 20000개의 자기 자신을 포함한 20000개의 문서 벡터 간의 유사도가 기록된 행렬임 

기존 데이터 프레임으로부터 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리를 만들면

In [28]:
title_to_index = dict(zip(data['title'], data.index))

idx = title_to_index['Father of the Bride Part II']
print(idx)

4


선택한 영화의 제목을 입력하면 코사인 유사도로 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만들면

In [30]:
def get_recommendations(title, cosine_sim=cosine_sim):
  idx = title_to_index[title]

  # 해당 영화와 모든 영화와의 유사도를 가져옴
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 영화들을 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 가장 유사한 10개의 영화를 받아옴
  # sim_scores[0]은 자기 자신이기 때문에 제외
  sim_scores = sim_scores[1:11]

  # 가장 유사한 10개의 영화의 인덱스를 얻음
  movie_indices = [idx[0] for idx in sim_scores]

  # 가장 유사한 10개의 영화의 제목을 리턴
  return data['title'].iloc[movie_indices]


  

영화 다크 나이트 라이즈와 overview가 유사한 영화를 찾아보면

In [31]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object